# Useless Information: QR Codes

A brief walkthrough on how QR codes are generated.

<style>
  table, td, th {
    border: 1px solid gray;
  }
  th {
    padding-top: 5px;
    padding-right: 10px;
    padding-bottom: 5px;
    padding-left: 10px;
  }
</style>

## Mathematics Refresher

<br>

**Monomial**: $3x^2$ (single term)

**Polynomial**: $4x^6+16x^4+2x^2+5x+1$ (multiple terms or monomials)

<br>

### Logarithms 

$y = \log_b x \implies b^y=x$

Example: $\log_2(8) = 3 \implies 2^3=8$

<br>

#### Multiplication via logarithms - A trivial example with base 2

$128 \times 512 = ?$

$\log_2(128) = 7,\; \log_2(512) = 9$

$128 \times 512 = 2^7 \times 2^9 = 2^{16} = 65536$

<br>
<img src="https://www.math.utah.edu/~alfeld/sliderules/side2half.jpg" alt="A slide rule" width="500" height="150"/>

This is how you multiply on a slide rule!

## Some Fun Facts About QR Codes

(Also known as, I skimmed the wikipedia page - https://en.wikipedia.org/wiki/QR_code)

- QR code -> Quick Response code
- 1994 Masahiro Hara at Denso Wave (Japanese automotive company)
- A matrix barcode (2D barcode) -> faster to read and stores more data
- Error correction (Reed-Solomon) allows damaged QR Codes to still be read

<br>

<img src="https://www.thonky.com/qr-code-tutorial/hello-world-final.png" alt="Hello World QR Code" width="250" height="250"/>
&nbsp;&nbsp;&nbsp;
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/QR_Code_Damaged.jpg/800px-QR_Code_Damaged.jpg" alt="Damaged QR Code" width="250" height="250"/>

<hr>

## Let's Make a QR Code

### Choose an Encoding Mode

| Mode         | Maximum Character (40-L) | Mode Indicator |
| ------------ | ------------------------ | -------------- |
| Numeric      | 7089 characters          | 0001           |
| Alphanumeric | 4296 characters          | 0010           |
| Byte         | 2953 characters          | 0100           |
| Kanji        | 1817 characters          | 0111           |

In [276]:
MODE_NUMERIC = 1   # 0001
MODE_ALPHANUM = 2  # 0010
MODE_BYTE = 4      # 0100
MODE_KANJI = 8     # 1000

In [277]:
# Declare payload and encode it (byte mode)

# UTF-8 encode -> hex bytes -> 8-bit binary
def encode_byte_mode(s):
    as_hex = [c.encode('utf-8').hex() for c in s]
    return [bin(int(byte, 16))[2:].zfill(8) for byte in as_hex]

# convert integer to bits
def int_to_bits(i, word_size):
    return bin(int(hex(i), 16))[2:].zfill(word_size)

payload = 'https://github.com/barrettotte'
encoded = encode_byte_mode(payload)
encoded_len = len(encoded)
mode = int_to_bits(MODE_BYTE, 4)

print(f"encoded '{payload}' to\n\n{encoded}\n\nsize: {encoded_len} byte(s)")
print([hex(ord(b)) for b in payload])
print(f'\nmode: {mode}')

encoded 'https://github.com/barrettotte' to

['01101000', '01110100', '01110100', '01110000', '01110011', '00111010', '00101111', '00101111', '01100111', '01101001', '01110100', '01101000', '01110101', '01100010', '00101110', '01100011', '01101111', '01101101', '00101111', '01100010', '01100001', '01110010', '01110010', '01100101', '01110100', '01110100', '01101111', '01110100', '01110100', '01100101']

size: 30 byte(s)
['0x68', '0x74', '0x74', '0x70', '0x73', '0x3a', '0x2f', '0x2f', '0x67', '0x69', '0x74', '0x68', '0x75', '0x62', '0x2e', '0x63', '0x6f', '0x6d', '0x2f', '0x62', '0x61', '0x72', '0x72', '0x65', '0x74', '0x74', '0x6f', '0x74', '0x74', '0x65']

mode: 0100


### Choose an Error Correction Level

Higher error correction, less character capacity

| Level | Name     | Recovery | Level Indicator |
| ----- | -------- | -------- | --------------- |
| L     | Low      | 7%       | 01              |
| M     | Medium   | 15%      | 00              |
| Q     | Quartile | 25%      | 11              |
| H     |  High    | 30%      | 10              |

In [278]:
ERROR_L = 1  # 01
ERROR_M = 0  # 00
ERROR_Q = 3  # 11
ERROR_H = 2  # 10
err_lvl = ERROR_Q

### Find the Version

Versions 1-40, select smallest possible!

| Version + Error Correction Level | Numeric | Alphanumeric | Byte | Kanji |
| -------------------------------- | ------- | ------------ | ---- | ----- |
| 1-L                              | 41      | 25           | 17   | 10    |
| 1-M                              | 34      | 20           | 14   | 8     |
| 1-Q                              | 27      | 16           | 11   | 7     |
| 1-H                              | 17      | 10           | 7    | 4     |
| ...                              | ...     | ...          | ...  | ...   |
| 3-L                              | 127     | 77           | 53   | 32    |
| 3-M                              | 101     | 61           | 42   | 26    |
| 3-Q                              | 77      | 47           | 32   | 20    |
| 3-H                              | 58      | 35           | 24   | 15    |
| ...                              | ...     | ...          | ...  | ...   |
| 40-L                             | 7089    | 4296         | 2953 | 1817  |
| 40-M                             | 5596    | 3391         | 2331 | 1435  |
| 40-Q                             | 3993    | 2420         | 1663 | 1024  |
| 40-H                             | 3057    | 1852         | 1273 | 784   |


In [279]:
# trimmed down version of capacity table
BYTE_MODE_CAPACITY_LOOKUP = [
    # L, M, Q, H
    [0, 0, 0, 0],       # (one-indexing)
    [17, 14, 11, 7],    # 1
    [32, 26, 20, 14],   # 2
    [53, 42, 32, 24],   # 3
    [78, 62, 46, 34],   # 4
    [106, 84, 60, 44],  # 5
    # and so on...to 40
]

# fixes issue with LMQH ordering
ERROR_IDX_TO_LOOKUP = [1, 0, 3, 2]

# find version to use based on payload size and error correction
def get_version(size, err_lvl):
    err_idx = ERROR_IDX_TO_LOOKUP[err_lvl]
    for col, row in enumerate(BYTE_MODE_CAPACITY_LOOKUP):
        if row[err_idx] > size:
            return col
    raise Exception("couldn't find version")

# find smallest version for our payload
version = get_version(encoded_len, err_lvl)
assert version == 3  # should be using a 3-Q configuration

print(f'payload is {encoded_len} character(s)')
print(f'selected version {version}')

payload is 30 character(s)
selected version 3


### Fetch Error Correction Configuration

The scheme used to break up our encoded bytes into groups/blocks to run through error correction.

In [280]:
EC_CONFIG_LOOKUP = [
    [],  # L                      M                       Q                    H
    [[19, 7, 1, 19, 0, 0], [16, 10, 1, 16, 0, 0], [13, 13, 1, 13, 0, 0], [9, 17, 1, 9, 0, 0]],         # 1
    [[34, 10, 1, 34, 0, 0], [28, 16, 1, 28, 0, 0], [22, 22, 1, 22, 0, 0], [16, 28, 1, 16, 0, 0]],      # 2
    [[55, 15, 1, 55, 0, 0], [44, 26, 1, 44, 0, 0], [34, 18, 2, 17, 0, 0], [26, 22, 2, 13, 0, 0]],      # 3
    [[80, 20, 1, 80, 0, 0], [64, 18, 2, 32, 0, 0], [48, 26, 2, 24, 0, 0], [36, 16, 4, 9, 0, 0]],       # 4
    [[108, 26, 1, 108, 0, 0], [86, 24, 2, 43, 0, 0], [62, 18, 2, 15, 2, 16], [46, 22, 2, 11, 2, 12]],  # 5
    # and so on...to 40
]

def get_ec_config(version, err_lvl):
    return EC_CONFIG_LOOKUP[version][ERROR_IDX_TO_LOOKUP[err_lvl]]

# fetch error correction configuration
ec_config = get_ec_config(version, err_lvl)
capacity = ec_config[0]
capacity_bits = capacity * 8

print('error correction config:')
print(f'  Total data words                           = {capacity}')
print(f'  Error correction words per block           = {ec_config[1]}')
print(f'  Number of blocks in group 1                = {ec_config[2]}')
print(f'  Number of data words in each group 1 block = {ec_config[3]}')
print(f'  Number of blocks in group 2                = {ec_config[4]}')
print(f'  Number of data words in each group 2 block = {ec_config[5]}')

error correction config:
  Total data words                           = 34
  Error correction words per block           = 18
  Number of blocks in group 1                = 2
  Number of data words in each group 1 block = 17
  Number of blocks in group 2                = 0
  Number of data words in each group 2 block = 0


## Fetch Character Count Indicator

Depending on version and encoding mode, the payload size will need to take up more bits.

| Version Range | Numeric | Alphanumeric | Byte    | Kanji   |
| ------------- | ------- | ------------ | ------- | ------- |
| 1-9           | 10 bits | 9 bits       | 8 bits  | 8 bits  |
| 10-26         | 12 bits | 11 bits      | 16 bits | 10 bits |
| 27-40         | 14 bits | 13 bits      | 16 bits | 12 bits |

In [281]:
# is test between low and high (inclusive)?
def is_between(low, high, test):
    return test >= low and test <= high

# determine character count indicator
def get_count(size, version, mode):
    if int(mode, 2) == MODE_BYTE:
        if is_between(1, 9, version):
            word_size = 8
        elif is_between(10, 40, version):
            word_size = 16
        else:
            raise Exception("Invalid version")
    else:
        raise Exception("Only byte mode implemented!")
    return int_to_bits(size, word_size)

count = get_count(encoded_len, version, mode)

print(f"size: {encoded_len} byte(s) -  == { hex(int(count, 2))} char count: {count}")
print(f"version {version} with max capacity of {capacity} byte(s)", end='')
print(f" or {capacity_bits} bit(s)")

size: 30 byte(s) -  == 0x1e char count: 00011110
version 3 with max capacity of 34 byte(s) or 272 bit(s)


## Pad the Payload

Before feeding the encoded payload into the error correction algorithm, it needs to be byte/bit padded.

In [282]:
# utility to build string of byte/bit size
def byte_size_str(d):
    size = len(d)
    return f"{size} bit(s) => {size // 8} byte(s), {size % 8} bit(s)"

seg = mode + count + ''.join(encoded)
print("seg: {0:0>4X}".format(int(seg, 2)))
print('before padding: ' + byte_size_str(seg))

# Add terminator of zeros up to four bits (if there is room)
terminal_bits = 0
while terminal_bits < 4 and len(seg) < capacity_bits:
    seg += '0'
    terminal_bits += 1

# pad bits to nearest byte
while len(seg) % 8 != 0 and len(seg) < capacity_bits:
    seg += '0'

# pad bytes to full capacity (alternating 0xEC and 0x11)
use_EC = True
while len(seg) < capacity_bits:
    seg += int_to_bits(int(0xEC), 8) if use_EC else int_to_bits(int(0x11), 8)
    use_EC = not use_EC

print("seg: {0:0>4X}".format(int(seg, 2)))
print(f'\nafter padding:  {byte_size_str(seg)}')
assert len(seg) == capacity_bits

seg: 41E68747470733A2F2F6769746875622E636F6D2F626172726574746F747465
before padding: 252 bit(s) => 31 byte(s), 4 bit(s)
seg: 41E68747470733A2F2F6769746875622E636F6D2F626172726574746F7474650EC11

after padding:  272 bit(s) => 34 byte(s), 0 bit(s)


## Split the Payload

Using error correction configuration, split the payload into groups and blocks

In [283]:
# split segment into words (bytes)
code_words = [seg[i: i + 8] for i in range(0, len(seg), 8)]
print(f'total word(s) = {len(code_words)}')

g1_blocks = []  # only two groups
g2_blocks = []  # so we can be lazy

ecw_per_block = ec_config[1]
g1_block_count = ec_config[2]
g1_data_block_size = ec_config[3]
g2_block_count = ec_config[4]
g2_data_block_size = ec_config[5]

print('\nerror correction config:')
print(f'  Total data words                           = {capacity}')
print(f'  Error correction words per block           = {ecw_per_block}')
print(f'  Number of blocks in group 1                = {g1_block_count}')
print(f'  Number of data words in each group 1 block = {g1_data_block_size}')
print(f'  Number of blocks in group 2                = {g2_block_count}')
print(f'  Number of data words in each group 2 block = {g2_data_block_size}\n')

# build group 1
cw_idx = 0
while len(g1_blocks) < g1_block_count:
    to_idx = g1_data_block_size * (len(g1_blocks) + 1)
    g1_blocks.append(code_words[cw_idx: to_idx])
    cw_idx += g1_data_block_size
assert len(g1_blocks) == g1_block_count

print(f'group 1 blocks:')
for i, b in enumerate(g1_blocks):
    print(f'\nblock {i}: {b}')

# build group 2
g2_offset = cw_idx
while len(g2_blocks) < g2_block_count:
    to_idx = (g2_data_block_size * (len(g2_blocks) + 1)) + g2_offset
    g2_blocks.append(code_words[cw_idx:to_idx])
    cw_idx += g2_data_block_size
assert len(g2_blocks) == g2_block_count

print(f'\ngroup 2 blocks:')
for i, b in enumerate(g2_blocks):
    print(f'\nblock {i}: {b}')

total word(s) = 34

error correction config:
  Total data words                           = 34
  Error correction words per block           = 18
  Number of blocks in group 1                = 2
  Number of data words in each group 1 block = 17
  Number of blocks in group 2                = 0
  Number of data words in each group 2 block = 0

group 1 blocks:

block 0: ['01000001', '11100110', '10000111', '01000111', '01000111', '00000111', '00110011', '10100010', '11110010', '11110110', '01110110', '10010111', '01000110', '10000111', '01010110', '00100010', '11100110']

block 1: ['00110110', '11110110', '11010010', '11110110', '00100110', '00010111', '00100111', '00100110', '01010111', '01000111', '01000110', '11110111', '01000111', '01000110', '01010000', '11101100', '00010001']

group 2 blocks:


## Reed-Solomon Error Correction

An error correction algorithm to allow a damaged payload to still get read correctly.

#### A Very High Level Overview of Galois Fields

A Galois or finite field is a field consisting of a finite amount of elements.

A finite field with $p^n$ elements is given by $\text{GF}(p^n)$, where $p$ is a prime number.

Think of a wagon wheel with $p^n$ spokes.

When $p$ is 2, we can start thinking in binary.
$\text{GF}(2^3) = (0,1,2,3,4,5,6,7)$

- $\text{GF}(8)[0] = 0(2^2) + 0(2^1) + 0(2^0) = 000$
- $\text{GF}(8)[1] = 0(2^2) + 0(2^1) + 1(2^0) = 001$
- $\text{GF}(8)[2] = 0(2^2) + 1(2^1) + 0(2^0) = 010$
- $\text{GF}(8)[3] = 0(2^2) + 1(2^1) + 1(2^0) = 011$
- $\text{GF}(8)[4] = 1(2^2) + 0(2^1) + 0(2^0) = 100$
- $\text{GF}(8)[5] = 1(2^2) + 0(2^1) + 1(2^0) = 101$
- $\text{GF}(8)[6] = 1(2^2) + 1(2^1) + 0(2^0) = 110$
- $\text{GF}(8)[7] = 1(2^2) + 1(2^1) + 1(2^0) = 111$

So, any binary number can be represented as a polynomial and vice versa.

$156=10011100 = 1x^7 + 0x^6 + 0x^5 + 1x^4 + 1x^3 + 1x^2 + 0x^1 + 0x^0 = x^7+x^4+x^3+x^2$

<br>

Finite fields are used in cryptography algorithms since they allow bytes to be easily 
and rapidly scrambled using polynomial arithmetic.

Reed-Solomon error correction uses $\text{GF}(2^8) = \text{GF}(256)$.

#### Finite Field Arithmetic in $\text{GF}(256)$

**Addition and subtraction**:

$(x^6+x^4+x+1) + (x^7+x^6+x^3+x) = x^7+x^4+x^3+1\;\;(\text{Coefficients are in GF(2))}$

$01010011 + 11001010 = 10011001 \implies \text{XOR}$

**Multiplication**:

I cheat and use a lookup table. But, the algorithm is called [Russian peasant multiplication](https://en.wikipedia.org/wiki/Ancient_Egyptian_multiplication#Russian_peasant_multiplication) which is actually a special case of the algorithm
used in ancient Egyptian multiplication.

In [284]:
# Add Galois functions
GF256_SIZE = 256

GF256_ANTILOG = [
    1, 2, 4, 8, 16, 32, 64, 128, 29, 58,               # 0 - 9
    116, 232, 205, 135, 19, 38, 76, 152, 45, 90,       # 10 - 19
    180, 117, 234, 201, 143, 3, 6, 12, 24, 48,         # 20 - 29
    96, 192, 157, 39, 78, 156, 37, 74, 148, 53,        # 30 - 39
    106, 212, 181, 119, 238, 193, 159, 35, 70, 140,    # 40 - 49
    5, 10, 20, 40, 80, 160, 93, 186, 105, 210,         # 50 - 59
    185, 111, 222, 161, 95, 190, 97, 194, 153, 47,     # 60 - 69
    94, 188, 101, 202, 137, 15, 30, 60, 120, 240,      # 70 - 79
    253, 231, 211, 187, 107, 214, 177, 127, 254, 225,  # 80 - 89
    223, 163, 91, 182, 113, 226, 217, 175, 67, 134,    # 90 - 99
    17, 34, 68, 136, 13, 26, 52, 104, 208, 189,        # 100 - 109
    103, 206, 129, 31, 62, 124, 248, 237, 199, 147,    # 110 - 119
    59, 118, 236, 197, 151, 51, 102, 204, 133, 23,     # 120 - 129
    46, 92, 184, 109, 218, 169, 79, 158, 33, 66,       # 130 - 139
    132, 21, 42, 84, 168, 77, 154, 41, 82, 164,        # 140 - 149
    85, 170, 73, 146, 57, 114, 228, 213, 183, 115,     # 150 - 159
    230, 209, 191, 99, 198, 145, 63, 126, 252, 229,    # 160 - 169
    215, 179, 123, 246, 241, 255, 227, 219, 171, 75,   # 170 - 179
    150, 49, 98, 196, 149, 55, 110, 220, 165, 87,      # 180 - 189
    174, 65, 130, 25, 50, 100, 200, 141, 7, 14,        # 190 - 199
    28, 56, 112, 224, 221, 167, 83, 166, 81, 162,      # 200 - 209
    89, 178, 121, 242, 249, 239, 195, 155, 43, 86,     # 210 - 219
    172, 69, 138, 9, 18, 36, 72, 144, 61, 122,         # 220 - 229
    244, 245, 247, 243, 251, 235, 203, 139, 11, 22,    # 230 - 239
    44, 88, 176, 125, 250, 233, 207, 131, 27, 54,      # 240 - 249
    108, 216, 173, 71, 142, 1                          # 250 - 255
]
GF256_LOG = [
    -1, 0, 1, 25, 2, 50, 26, 198, 3, 223,              # 0 - 9
    51, 238, 27, 104, 199, 75, 4, 100, 224, 14,        # 10 - 19
    52, 141, 239, 129, 28, 193, 105, 248, 200, 8,      # 20 - 29
    76, 113, 5, 138, 101, 47, 225, 36, 15, 33,         # 30 - 39
    53, 147, 142, 218, 240, 18, 130, 69, 29, 181,      # 40 - 49
    194, 125, 106, 39, 249, 185, 201, 154, 9, 120,     # 50 - 59
    77, 228, 114, 166, 6, 191, 139, 98, 102, 221,      # 60 - 69
    48, 253, 226, 152, 37, 179, 16, 145, 34, 136,      # 70 - 79
    54, 208, 148, 206, 143, 150, 219, 189, 241, 210,   # 80 - 89
    19, 92, 131, 56, 70, 64, 30, 66, 182, 163,         # 90 - 99
    195, 72, 126, 110, 107, 58, 40, 84, 250, 133,      # 100 - 109
    186, 61, 202, 94, 155, 159, 10, 21, 121, 43,       # 110 - 119
    78, 212, 229, 172, 115, 243, 167, 87, 7, 112,      # 120 - 129
    192, 247, 140, 128, 99, 13, 103, 74, 222, 237,     # 130 - 139
    49, 197, 254, 24, 227, 165, 153, 119, 38, 184,     # 140 - 149
    180, 124, 17, 68, 146, 217, 35, 32, 137, 46,       # 150 - 159
    55, 63, 209, 91, 149, 188, 207, 205, 144, 135,     # 160 - 169
    151, 178, 220, 252, 190, 97, 242, 86, 211, 171,    # 170 - 179
    20, 42, 93, 158, 132, 60, 57, 83, 71, 109,         # 180 - 189
    65, 162, 31, 45, 67, 216, 183, 123, 164, 118,      # 190 - 199
    196, 23, 73, 236, 127, 12, 111, 246, 108, 161,     # 200 - 209
    59, 82, 41, 157, 85, 170, 251, 96, 134, 177,       # 210 - 219
    187, 204, 62, 90, 203, 89, 95, 176, 156, 169,      # 220 - 229
    160, 81, 11, 245, 22, 235, 122, 117, 44, 215,      # 230 - 239
    79, 174, 213, 233, 230, 231, 173, 232, 116, 214,   # 240 - 249
    244, 234, 168, 80, 88, 175                         # 250 - 255
]

def gf256_add(a: int, b: int):
    return a ^ b

def gf256_sub(a: int, b: int):
    return gf256_add(a, b)

def gf256_mul(a: int, b: int):
    return 0 if (a == 0 or b == 0) else GF256_ANTILOG[(GF256_LOG[a] + GF256_LOG[b]) % (GF256_SIZE - 1)]

def gf256_inv(a: int):
    if a == 0:
        raise Exception("Zero has no inverse")
    return GF256_ANTILOG[(GF256_SIZE - 1) - GF256_LOG[a]]

def gf256_div(a: int, b: int):
    if a == 0:
        return 0
    elif b == 0:
        raise Exception("div by zero in GF")
    return gf256_mul(a, gf256_inv(b))

In [285]:
# add polynomial functions

class Polynomial():

    def __init__(self, terms: list):
        self.terms = terms

    def __str__(self):
        return ' + '.join([f"{t}x^{len(self.terms) - i - 1}" for i, t in enumerate(self.terms[::-1]) if t > 0])

    # return __str__ in alpha notation
    def str_alpha(self):
        return ' + '.join([f"α^{GF256_LOG[t]}x^{len(self.terms) - i - 1}" for i, t in enumerate(self.terms[::-1]) if t > 0])

    # return degree of polynomial
    def get_degree(self):
        return len(self.terms) - 1

    # determine if two polynomials are equivalent
    def equals(self, other):
        if len(self.terms) > len(other.terms):
            min_poly = other
            max_poly = self
        else:
            min_poly = self
            max_poly = other
        for i in range(len(min_poly.terms)):
            if self.terms[i] != other.terms[i]:
                return False
        for i in range(len(min_poly.terms), len(max_poly.terms)):
            if max_poly.terms[i] != 0:
                return False
        return True

# create new polynomial from a block of words
# each word becomes the coefficient of an x term
def block_to_poly(block: list):
    terms = ([int(w, 2) for w in block])[::-1]
    return Polynomial(terms)

# multiply polynomial by alpha value
def poly_alpha_mul(p: Polynomial, alpha: int):
    for i, t in enumerate(p.terms):
        t_alpha = (GF256_LOG[t] + alpha) % (GF256_SIZE - 1)
        p.terms[i] = GF256_ANTILOG[t_alpha]
    return p

# normalize polynomial
def poly_normalize(p: Polynomial):
    # print(f"[poly_norm] normalizing {p}   [{len(p.terms)}]")
    max_nz = len(p.terms) - 1  # max nonzero term
    for i in range(len(p.terms) - 1, 0, -1):
        if p.terms[i] != 0: break
        max_nz = i - 1
    if max_nz < 0:
        # print(f"[poly_norm] normalized to {p}   [{len(p.terms)}] **** ????")
        return Polynomial([0])
    elif max_nz < (len(p.terms) - 1): 
        p.terms = p.terms[0: max_nz + 1]
    # print(f"[poly_norm] normalized to {p}   [{len(p.terms)}]")
    return p

# add two polynomials
def poly_add(a: Polynomial, b: Polynomial):
    # print(f"    [poly_add]  ({a}) + ({b})\n    [poly_add]  a[{len(a.terms)}] ; b[{len(b.terms)}]")
    term_len = len(a.terms)
    if len(b.terms) > term_len:
        term_len = len(b.terms)
    p = Polynomial([0] * term_len)
    for i in range(term_len):
        if len(a.terms) > i and len(b.terms) > i:
            p.terms[i] = gf256_add(a.terms[i], b.terms[i])
            # print(f"      [poly_add]  prdA_poly[{i}] = gf256_add({a.terms[i]}, {b.terms[i]}) = {p.terms[i]}")
        elif len(a.terms) > i:
            p.terms[i] = a.terms[i]
            # print(f"      [poly_add]  prdA_poly[{i}] = a[{i}] = {p.terms[i]}")
        else:
            p.terms[i] = b.terms[i]
            # print(f"      [poly_add]  prdA_poly[{i}] = b[{i}] = {p.terms[i]}")
    return poly_normalize(p)

# multiply two polynomials
def poly_mul(a: Polynomial, b: Polynomial):
    # print(f"   [poly_mul] a={a}\n b={b}")
    p = Polynomial([0] * (len(a.terms) + len(b.terms)))
    for i in range(len(a.terms)):
        for j in range(len(b.terms)):
            if a.terms[i] != 0 and b.terms[j] != 0:
                monomial = Polynomial([0] * (i + j + 1))
                monomial.terms[i + j] = gf256_mul(a.terms[i], b.terms[j])
                # print(f"  [poly_mul ({i},{j})] prdB_poly = {monomial}")
                # print(f"  [poly_mul] gf256_mul({a.terms[i]}, {b.terms[j]}) = {monomial.terms[i + j]}")
                p = poly_add(p, monomial)
                # print(f"  [poly_mul ({i},{j}) done]  a + b = prdA_poly = {p}\n")
    return poly_normalize(p)

# perform polynomial long division and return remainder polynomial
def poly_remainder(numerator: Polynomial, denominator: Polynomial):
    if numerator.equals(denominator):
        raise Exception("Remainder is zero")
    remainder = numerator

    while len(remainder.terms) >= len(denominator.terms):
        degree = len(remainder.terms) - len(denominator.terms)
        coefficient = gf256_div(remainder.terms[-1], denominator.terms[-1])
        m = new_monomial(coefficient, degree)
        # print(f"\n[poly_remainder]  denominator = [{len(denominator.terms)}] {denominator}")
        # print(f"[poly_remainder] mono = [{len(m.terms)}] {m}")
        divisor = poly_mul(denominator, m)
        # print(f"[poly_remainder]  = divisor = {divisor}\n")
        # print(f"[poly_remainder]  remainder + divisor = ({remainder}) + ({divisor})")
        remainder = poly_add(remainder, divisor)
        # print(f"[poly_remainder]  = remainder = {remainder}\n")
    return poly_normalize(remainder)

# create a monomial (single term polynomial) with given term and degree
def new_monomial(term: int, degree: int):
    if term == 0:
        return Polynomial([0])
    mono = Polynomial([0] * (degree + 1))
    mono.terms[degree] = term
    return mono

# create a polynomial from bit string
def bits_to_poly(bits: str):
    return Polynomial([int(bits[i]) for i in range(len(bits))])

# create bit string from polynomial
def poly_to_bits(p: Polynomial):
    return ''.join(['1' if t > 0 else '0' for t in p.terms])

In [286]:
# calculate error correction blocks with Reed-Solomon 

ec_blocks = []

def get_gen_poly(degree: int):
    if degree < 2:
        raise Exception('generator polynomial degree must be greater than 2')
    gp = Polynomial([1])
    for i in range(degree):
        np = Polynomial([GF256_ANTILOG[i], 1])
        # print(f"\n========={i}==========\ngp = {gp}")
        # print(f"np = {np}\ngp = gp * np = ({gp}) * ({np})")
        gp = poly_mul(gp, np)
        # print(f"gp = {gp}")
        # terms = '   '.join([str(t[1]).rjust(3) for t in enumerate(gp.terms)])
        # print(f"{str(i).zfill(2)}:  {str(len(gp.terms)).rjust(3)}   {terms}")
    print(".............................................")
    return gp

# group 1 error correction
for i, block in enumerate(g1_blocks):
    print(f'\nblock {i}')
    print([int(word, 2) for word in block])

    # translate block of data to message polynomial
    msg_poly = block_to_poly(block)
    print(f"\nmsg = {msg_poly}\n")

    # build generator polynomial
    gen_poly = get_gen_poly(ecw_per_block)
    print(f"gen = {gen_poly}\n")

    # ensure lead term doesn't become too small during division
    mono = new_monomial(1, ecw_per_block)
    # print(f"msg_poly = [{len(msg_poly.terms)}] {msg_poly}\nmono = [{len(mono.terms)}] {mono}")
    msg_poly = poly_mul(msg_poly, mono)
    # print(f"msg_poly * mono = {msg_poly}\n")

    # find error correction words via polynomial long division
    print(f"******************")
    rem_poly = poly_remainder(msg_poly, gen_poly)
    print(f"remainder of msg / gen = {rem_poly}\n")
    ec_block = [int_to_bits(word, 8) for word in rem_poly.terms[::-1]]

    print(f"{len(ec_block)} error correction words:\n{[hex(int(word, 2)) for word in ec_block]}")
    print(f"{[int(word, 2) for word in ec_block]}")
    assert len(ec_block) == ecw_per_block
    ec_blocks.append(ec_block)


block 0
[65, 230, 135, 71, 71, 7, 51, 162, 242, 246, 118, 151, 70, 135, 86, 34, 230]

msg = 65x^16 + 230x^15 + 135x^14 + 71x^13 + 71x^12 + 7x^11 + 51x^10 + 162x^9 + 242x^8 + 246x^7 + 118x^6 + 151x^5 + 70x^4 + 135x^3 + 86x^2 + 34x^1 + 230x^0

.............................................
gen = 1x^18 + 239x^17 + 251x^16 + 183x^15 + 113x^14 + 149x^13 + 175x^12 + 199x^11 + 215x^10 + 240x^9 + 220x^8 + 73x^7 + 82x^6 + 173x^5 + 75x^4 + 32x^3 + 67x^2 + 217x^1 + 146x^0

******************
remainder of msg / gen = 253x^17 + 51x^16 + 17x^15 + 168x^14 + 54x^13 + 42x^12 + 245x^11 + 204x^10 + 174x^9 + 233x^8 + 27x^7 + 198x^6 + 80x^5 + 35x^4 + 131x^3 + 242x^1 + 202x^0

18 error correction words:
['0xfd', '0x33', '0x11', '0xa8', '0x36', '0x2a', '0xf5', '0xcc', '0xae', '0xe9', '0x1b', '0xc6', '0x50', '0x23', '0x83', '0x0', '0xf2', '0xca']
[253, 51, 17, 168, 54, 42, 245, 204, 174, 233, 27, 198, 80, 35, 131, 0, 242, 202]

block 1
[54, 246, 210, 246, 38, 23, 39, 38, 87, 71, 70, 247, 71, 70, 80, 236, 17]


In [287]:
# interleave payload data and error correction data
data = []
for i in range(g1_data_block_size):
    for j in range(len(g1_blocks)):
        data.append(g1_blocks[j][i])
for i in range(ecw_per_block):
    for j in range(len(ec_blocks)):
        data.append(ec_blocks[j][i])
print(f"payload = '{payload}'\n")
print(f"interleaved data - {len(data)} word(s):\n{[hex(int(x, 2)) for x in data]}")

payload = 'https://github.com/barrettotte'

interleaved data - 70 word(s):
['0x41', '0x36', '0xe6', '0xf6', '0x87', '0xd2', '0x47', '0xf6', '0x47', '0x26', '0x7', '0x17', '0x33', '0x27', '0xa2', '0x26', '0xf2', '0x57', '0xf6', '0x47', '0x76', '0x46', '0x97', '0xf7', '0x46', '0x47', '0x87', '0x46', '0x56', '0x50', '0x22', '0xec', '0xe6', '0x11', '0xfd', '0x9', '0x33', '0x6b', '0x11', '0x1e', '0xa8', '0x76', '0x36', '0x15', '0x2a', '0x6c', '0xf5', '0xe3', '0xcc', '0xf', '0xae', '0x75', '0xe9', '0x8b', '0x1b', '0xf', '0xc6', '0xb2', '0x50', '0x8e', '0x23', '0x4f', '0x83', '0x97', '0x0', '0xa2', '0xf2', '0xc8', '0xca', '0x39']


In [288]:
# add remainder bits based on version
REMAINDER_LOOKUP = [
    0,  # one indexing
    0, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3,
    4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0
]
remainder_bits = REMAINDER_LOOKUP[version]
print(f"Adding {remainder_bits} remainder bit(s)")
data = ''.join([x for x in data]) + ('0' * remainder_bits)
print(f"data - ({len(data)}) bit(s)\n\n{data}")

Adding 7 remainder bit(s)
data - (567) bit(s)

010000010011011011100110111101101000011111010010010001111111011001000111001001100000011100010111001100110010011110100010001001101111001001010111111101100100011101110110010001101001011111110111010001100100011110000111010001100101011001010000001000101110110011100110000100011111110100001001001100110110101100010001000111101010100001110110001101100001010100101010011011001111010111100011110011000000111110101110011101011110100110001011000110110000111111000110101100100101000010001110001000110100111110000011100101110000000010100010111100101100100011001010001110010000000


In [289]:
# start building QR matrix
qr_size = ((version - 1) * 4) + 21
qr_mat = [2] * (qr_size ** 2)  # flat list (so ASM port is easier)
print(f"QR matrix : {qr_size} x {qr_size} - {len(qr_mat)} module(s)\n")

# save matrix to file
from PIL import Image

def mat_to_file(qr_mat, qr_size, file_name, normalize=False):
    to_print = qr_mat
    if normalize:
        to_print = normalize_matrix(to_print, qr_size)
    img = Image.new(mode='1', size=(qr_size, qr_size))

    to_print = [not b for b in to_print]  # reverse bits
    for x in range(qr_size):
        for y in range(qr_size):
            pixel = to_print[(y * qr_size) + x]
            img.putpixel((x, y), pixel)
    img.save(file_name)

def normalize_matrix(qr_mat, qr_size):
    n = []
    for x in range(qr_size):
        for y in range(qr_size):
            n.append([0, 1, 0, 0, 1][qr_mat[(y * qr_size) + x]])
    return n

mat_to_file(qr_mat, qr_size, 'qrcode.png', normalize=True)

QR matrix : 29 x 29 - 841 module(s)



In [290]:
# add reserved areas

# draw square in qr matrix
def draw_square(qr_mat, qr_size, x, y, n, c):
    for i in range(n):
        for j in range(n):
            dx = (x + i)
            dy = (y + j)
            if dx < qr_size and dy < qr_size and dx >= 0 and dy >= 0:
                qr_mat[(dy * qr_size) + dx] = c
    return qr_mat

if version > 6:
    raise Exception("QR versions greater than 6 are not supported")
qr_mat = draw_square(qr_mat, qr_size, 0, 0, 9, 3)                  # top left
qr_mat = draw_square(qr_mat, qr_size, (qr_size - 7) - 1, 0, 9, 3)  # top right
qr_mat = draw_square(qr_mat, qr_size, 0, (qr_size - 7), 9, 3)      # bottom left
mat_to_file(qr_mat, qr_size, 'qrcode.png', normalize=True)

In [291]:
# place timing patterns

# horizontal timing
is_fill = True
for i in range(qr_size):
    c = 4 if is_fill else 3
    qr_mat[(i) * qr_size + (6)] = c
    is_fill = not is_fill

# vertical timing
is_fill = True
for j in range(qr_size):
    c = 4 if is_fill else 3
    qr_mat[(6) * qr_size + (j)] = c
    is_fill = not is_fill

mat_to_file(qr_mat, qr_size, 'qrcode.png', normalize=True)

In [292]:
# place finder pattern in QR code matrix with left corner at (x,y)
def place_finder(qr_mat, qr_size, x, y):
    qr_mat = draw_square(qr_mat, qr_size, x - 1, y - 1, 9, 3)  # separator
    qr_mat = draw_square(qr_mat, qr_size, x, y, 7, 4)          # outer
    qr_mat = draw_square(qr_mat, qr_size, x + 1, y + 1, 5, 3)  # inner
    qr_mat = draw_square(qr_mat, qr_size, x + 2, y + 2, 3, 4)  # center
    return qr_mat

qr_mat = place_finder(qr_mat, qr_size, 0, 0)              # top left
qr_mat = place_finder(qr_mat, qr_size, 0, (qr_size - 7))  # top right
qr_mat = place_finder(qr_mat, qr_size, (qr_size - 7), 0)  # bottom left

mat_to_file(qr_mat, qr_size, 'qrcode.png', normalize=True)

In [293]:
# add misc reserved areas
ALIGNMENT_PATTERN_LOOK = [
    [],  # one indexing
    [],  # version 1 has no alignment
    [6, 18, 0, 0, 0, 0, 0], [6, 22, 0, 0, 0, 0, 0],      # 2, 3
    [6, 26, 0, 0, 0, 0, 0], [6, 30, 0, 0, 0, 0, 0],      # 4, 5
    [6, 34, 0, 0, 0, 0, 0], [6, 22, 38, 0, 0, 0, 0],     # 6, 7
    # and so on to 40...
]

# place alignment pattern
if version > 1:
    pat = ALIGNMENT_PATTERN_LOOK[version]
    qr_mat = draw_square(qr_mat, qr_size, pat[1] - 2, pat[1] - 2, 5, 4)
    qr_mat = draw_square(qr_mat, qr_size, pat[1] - 1, pat[1] - 1, 3, 3)
    qr_mat = draw_square(qr_mat, qr_size, pat[1], pat[1], 1, 4)

# place dark module
qr_mat[((4 * version) + 9) * qr_size + (8)] = 4
mat_to_file(qr_mat, qr_size, 'qrcode.png', normalize=True)

In [294]:
# "zigzag" the data
def zigzag_data(qr_mat, qr_size, data):
    x = qr_size - 1
    y = qr_size - 1
    data_idx = 0
    zig = True
    up = True

    while data_idx < len(data):
        # reached edge, bounce back
        if y == qr_size:
            up = not up
            x -= 2
            zig = True
            y = qr_size - 1
        elif y < 0:
            up = not up
            x -= 2
            zig = True
            y = 0
        next_mod = qr_mat[(y * qr_size) + x]

        # zig zag past existing structure
        if next_mod == 2:
            qr_mat[(y * qr_size) + x] = int(data[data_idx])
            data_idx += 1

        # zig or zag
        if zig:
            x -= 1
        else:
            y += 1 if not up else -1
            x += 1
        zig = not zig

        # skip over timing patterns
        if x == 6:
            y -= 1
            x -= 1
    return qr_mat

qr_mat = zigzag_data(qr_mat, qr_size, data)
mat_to_file(qr_mat, qr_size, 'qrcode.png', normalize=True)

In [295]:
# generate masks
def get_masks(qr_size):
    masks = []

    # mask 0
    mask = [0] * (qr_size ** 2)
    for y in range(qr_size):
        for x in range(qr_size):
            mask[(y * qr_size) + x] = 1 if ((x + y) % 2) == 0 else 0
    masks.append(mask)

    # mask 1
    mask = [0] * (qr_size ** 2)
    for y in range(qr_size):
        for x in range(qr_size):
            mask[(y * qr_size) + x] = 1 if (y % 2) == 0 else 0
    masks.append(mask)

    # mask 2
    mask = [0] * (qr_size ** 2)
    for y in range(qr_size):
        for x in range(qr_size):
            mask[(y * qr_size) + x] = 1 if (x % 3) == 0 else 0
    masks.append(mask)

    # mask 3
    mask = [0] * (qr_size ** 2)
    for y in range(qr_size):
        for x in range(qr_size):
            mask[(y * qr_size) + x] = 1 if ((x + y) % 3) == 0 else 0
    masks.append(mask)

    # mask 4
    mask = [0] * (qr_size ** 2)
    for y in range(qr_size):
        for x in range(qr_size):
            mask[(y * qr_size) + x] = 1 if ((x // 3 + y // 2) % 2) == 0 else 0
    masks.append(mask)

    # mask 5
    mask = [0] * (qr_size ** 2)
    for y in range(qr_size):
        for x in range(qr_size):
            mask[(y * qr_size) + x] = 1 if ((x * y % 2) + (x * y % 3)) == 0 else 0
    masks.append(mask)

    # mask 6
    mask = [0] * (qr_size ** 2)
    for y in range(qr_size):
        for x in range(qr_size):
            mask[(y * qr_size) + x] = 1 if (((x * y) % 2 + x * y % 3) % 2) == 0 else 0
    masks.append(mask)

    # mask 7
    mask = [0] * (qr_size ** 2)
    for y in range(qr_size):
        for x in range(qr_size):
            mask[(y * qr_size) + x] = 1 if (((x + y) % 2 + x * y % 3) % 2) == 0 else 0
    masks.append(mask)

    return masks

masks = get_masks(qr_size)
for i,mask in enumerate(masks):
    mat_to_file(mask, qr_size, f'mask-{i}.png')

In [296]:
# determine best mask
FMT_INFO_LOOKUP = [
    [    # L
        '111011111000100', '111001011110011', '111110110101010', '111100010011101',
        '110011000101111', '110001100011000', '110110001000001', '110100101110110'
    ],
    [    # M
        '101010000010010', '101000100100101', '101111001111100', '101101101001011',
        '100010111111001', '100000011001110', '100111110010111', '100101010100000'
    ],
    [
        # Q
        '011010101011111', '011000001101000', '011111100110001', '011101000000110',
        '010010010110100', '010000110000011', '010111011011010', '010101111101101'
    ],
    [   # H
        '001011010001001', '001001110111110', '001110011100111', '001100111010000',
        '000011101100010', '000001001010101', '000110100001100', '000100000111011'
    ]
]

# calculate format bits
def calc_fmt_bits(err_lvl, mask_idx):
    fmt_bits = int_to_bits(err_lvl, 2) + int_to_bits(mask_idx, 3)
    err_bits = (fmt_bits + ('0' * 10)).lstrip('0')

    # calculate error correction bits
    while len(err_bits) >= 11:
        # build generator polynomial
        res = ''
        gen_bits = '10100110111'  # $x^{10}+x^8+x^5+x^4+x^2+x+1$

        # pad generator polynomial to match length of format bits
        while len(gen_bits) != len(err_bits):
            gen_bits += '0'
        # XOR generator bits with format string
        for i in range(len(gen_bits)):
            res += str(int(gen_bits[i]) ^ int(err_bits[i]))
        err_bits = res.lstrip('0')

    # repad to 10-bits
    while len(err_bits) < 10:
        err_bits = '0' + err_bits

    # combine format and error correction bits
    fmt_bits += err_bits
    final_fmt_bits = ''
    for i in range(len(fmt_bits)):
        final_fmt_bits += str(int(fmt_bits[i]) ^ int('101010000010010'[i]))

    lookup_fmt = FMT_INFO_LOOKUP[ERROR_IDX_TO_LOOKUP[err_lvl]][mask_idx]
    assert final_fmt_bits == lookup_fmt
    return final_fmt_bits

# add format bits adjacent to finders
def add_format_bits(qr_mat, qr_size, fmt_bits):
    # break up format bits to place near finder patterns
    high_bits = fmt_bits[0:7]  # MSB=0
    low_bits = fmt_bits[8:15]  # LSB=14

    # top left format bits
    x = 0
    y = 8
    for i in range(len(high_bits)):
        if i == 6:
            x += 1  # skip vertical timing
        qr_mat[(y * qr_size) + x] = int(high_bits[i])
        x += 1
    x = 8
    y = 7
    for j in range(len(low_bits)):
        if j == 1:
            y -= 1  # skip horizontal timing
        qr_mat[(y * qr_size) + x] = int(low_bits[j])
        y -= 1

    # top right format bits
    x = qr_size - 7
    y = 8
    for i in range(len(low_bits)):
        qr_mat[(y * qr_size) + x + i] = int(low_bits[i])

    # bottom left format bits
    x = 8
    y = qr_size - 1
    for i in range(len(low_bits)):
        qr_mat[((y - i) * qr_size) + x] = int(high_bits[i])

    return qr_mat

# Evaluate penalty for rule 1: group of 5 or more same-colored modules in a row or col
def eval_rule_1(masked, qr_size):
    row_count = 0
    col_count = 0
    prev_row = 0
    prev_col = 0
    penalty_horizontal = 0
    penalty_vertical = 0
    module = -1

    for y in range(qr_size):
        if module == prev_col:
            col_count += 1
        else:
            col_count = 0

        if col_count == 5:
            penalty_vertical += 3
        elif col_count > 5:
            penalty_vertical += 1

        for x in range(qr_size):
            module = masked[(y * qr_size) + x]
            if module == prev_row:
                row_count += 1
            else:
                row_count = 0

            if row_count == 5:
                penalty_horizontal += 3
            elif row_count > 5:
                penalty_horizontal += 1
            prev_row = module
        row_count = 0
        prev_col = 0
    return penalty_horizontal + penalty_vertical

# Evaluate penalty for rule 2: 2x2 area of same colored modules
def eval_rule_2(masked, qr_size):
    penalty = 0
    for x in range(qr_size):
        for y in range(qr_size):
            idx = (y * qr_size) + x
            if (x < qr_size - 1) and (y < qr_size - 1) and (y > 0):
                is_square = True
                test = masked[idx]  # top left

                if test != masked[idx + 1]:
                    is_square = False  # top right
                elif test != masked[idx + qr_size]:
                    is_square = False  # bottom left
                elif test != masked[idx + qr_size + 1]:
                    is_square = False  # bottom right

                if is_square:
                    penalty += 3
    return penalty
                        
# Evaluate penalty for rule 3: occurrences of 10111010000 and 00001011101 in rows/cols
def eval_rule_3(masked, qr_size):
    return 0  # skipping this...could not get it working for some reason...

# Evaluate penalty for rule 4: ratio of light to dark modules
def eval_rule_4(masked, qr_size):
    white = 0
    black = 0
    for x in range(qr_size):
        for y in range(qr_size):
            idx = (y * qr_size) + x
            if masked[idx] == 1:
                black += 1
            else:
                white += 1
    total = white + black
    return ((abs(black * 20 - total * 10) + total - 1) // (total - 1)) * 10

# apply mask to QR matrix (not affecting non-function modules)
def apply_mask(mask, qr_mat, qr_size):
    masked = [0] * (qr_size ** 2)
    for y in range(qr_size):
        for x in range(qr_size):
            idx = (y * qr_size) + x
            module = qr_mat[idx]

            # 3-4 are reserved
            if module < 2:
                masked[idx] = module ^ mask[idx]
            elif module == 3:
                masked[idx] = 0  # swap out reserved '0'
            elif module == 4:
                masked[idx] = 1  # swap out reserved '1'
    return masked

# apply each mask and use penalty to determine most ideal
def apply_ideal_mask(qr_mat, qr_size, err_lvl):
    masks = get_masks(qr_size)
    min_penalty = 99999999
    ideal_mask_idx = -1

    for mask_idx, mask in enumerate(masks):
        penalty = 0
        fmt_bits = calc_fmt_bits(err_lvl, mask_idx)
        masked = add_format_bits(qr_mat, qr_size, fmt_bits)
        masked = apply_mask(mask, masked, qr_size)

        penalty += eval_rule_1(masked, qr_size)
        penalty += eval_rule_2(masked, qr_size)
        penalty += eval_rule_3(masked, qr_size)
        penalty += eval_rule_4(masked, qr_size)

        if penalty < min_penalty:
            min_penalty = penalty
            ideal_mask_idx = mask_idx
        print(f"mask {mask_idx} has penalty {penalty}")
    print(f"ideal mask is mask {ideal_mask_idx}")

    # apply ideal mask
    fmt_bits = calc_fmt_bits(err_lvl, ideal_mask_idx)
    masked = apply_mask(masks[ideal_mask_idx], qr_mat, qr_size)
    final_mat = add_format_bits(masked, qr_size, fmt_bits)
    return final_mat

qr_mat = apply_ideal_mask(qr_mat, qr_size, err_lvl)
mat_to_file(qr_mat, qr_size, 'qrcode.png')

mask 0 has penalty 396
mask 1 has penalty 418
mask 2 has penalty 351
mask 3 has penalty 318
mask 4 has penalty 347
mask 5 has penalty 334
mask 6 has penalty 377
mask 7 has penalty 291
ideal mask is mask 7


In [297]:
# add 4 module wide area of light modules (quiet zone)
def add_quiet_zone(qr_mat, qr_size):
    quieted = [0] * ((qr_size + 8) ** 2)
    for x in range(0, qr_size):
        for y in range(0, qr_size):
            module = qr_mat[(y * qr_size) + x]
            quieted[((y + 4) * (qr_size + 8)) + (x + 4)] = module
    return quieted

qr_mat = add_quiet_zone(qr_mat, qr_size)
mat_to_file(qr_mat, qr_size + 8, 'qrcode.png')